In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow import config
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
# from tokenization import return_tokens
# from julia import Main
import h5py
import hdf5plugin

In [2]:
config.list_physical_devices('GPU')

# https://stackoverflow.com/questions/41402409/tensorflow-doesnt-seem-to-see-my-gpu

[]

In [3]:
f = h5py.File(f'{os.getcwd()}//augmented_data.jld', 'r')
X = np.array(f['X'])
X = np.transpose(X)
Y = np.array(f['Y'])
Y = np.transpose(Y)

In [4]:
n = 60_000

indicies = pd.DataFrame(range(len(X)), columns=['i'])
indicies = indicies['i'].sample(n=n).to_numpy()

sampled_X = np.zeros((n, X.shape[1], X.shape[2]))
sampled_Y = np.zeros((n, Y.shape[1]))

for n, i in enumerate(indicies):
    sampled_X[n] = X[i]
    sampled_Y[n] = Y[i]

print(np.average([np.argmax(i) for i in sampled_Y]))

In [9]:
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=.1)
trainX.shape, testX.shape, trainY.shape, testY.shape

((54000, 190, 72), (6000, 190, 72), (54000, 2), (6000, 2))

In [10]:
opt = Adam(.0001)
input_shape = trainX[0].shape

In [11]:
model = Sequential()
model.add(LSTM(514, input_shape=input_shape, activation="tanh", return_sequences=True, dropout=0.2)) 
model.add(LSTM(256, activation="tanh", return_sequences=True, dropout=0.2))
model.add(LSTM(128, activation="tanh", dropout=0.2))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(16, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2, activation="softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 190, 128)          102912    
_________________________________________________________________
lstm_1 (LSTM)                (None, 190, 128)          131584    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0

In [12]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
history = model.fit(trainX, trainY, batch_size=32, epochs=100, validation_data=(testX, testY), verbose=1)

Epoch 1/100
1688/1688 [==============================] - 319s 189ms/step - loss: 0.6477 - accuracy: 0.6236 - val_loss: 0.6227 - val_accuracy: 0.6507
Epoch 2/100
1688/1688 [==============================] - 304s 180ms/step - loss: 0.6293 - accuracy: 0.6484 - val_loss: 0.5950 - val_accuracy: 0.6747
Epoch 3/100
1688/1688 [==============================] - 301s 178ms/step - loss: 0.6188 - accuracy: 0.6586 - val_loss: 0.5758 - val_accuracy: 0.7028
Epoch 4/100
1688/1688 [==============================] - 299s 177ms/step - loss: 0.6081 - accuracy: 0.6690 - val_loss: 0.5702 - val_accuracy: 0.7135
Epoch 5/100
1688/1688 [==============================] - 297s 176ms/step - loss: 0.6000 - accuracy: 0.6766 - val_loss: 0.5627 - val_accuracy: 0.7020
Epoch 6/100
1688/1688 [==============================] - 297s 176ms/step - loss: 0.5951 - accuracy: 0.6822 - val_loss: 0.5582 - val_accuracy: 0.7097
Epoch 7/100
1688/1688 [==============================] - 297s 176ms/step - loss: 0.5899 - accuracy: 0.6866

In [ ]:
hist_df = pd.DataFrame(history.history) 
hist_df.to_csv(f'{os.getcwd()}//model_history.csv')
model.save(f'{os.getcwd()}//rnn_model.h5')

In [ ]:
class CustomGen(tf.keras.utils.Sequence) :
    def __init__(self, X, Y, batch_size, length=None):
        self.X = X
        self.Y = Y
        self.batch_size = batch_size
        self.length = length

    def __len__(self):
        if self.length is None:
            self.length = len(X)

        return int(self.length / self.batch_size)
        
    def __getitem__(self, idx) :
        # xfile, yfile = self.filenames

        # xdf = pd.read_csv(xfile, sep="&", skiprows=idx*self.batch_size, nrows=self.batch_size, header=None)
        # xdf.columns = ['X']
        # ydf = pd.read_csv(yfile, skiprows=idx*self.batch_size, nrows=self.batch_size, header=None)
        # ydf.columns = ['Y']

        # if self.x_size == None:
        #     self.x_size = len(ast.literal_eval(xdf.iloc[0, 0]))

        # x = np.zeros((self.batch_size, self.x_size)) # must be seq length or len(ast.literal_eval(xdf[0][0])))
        # for n, i in enumerate(xdf.iloc[:, 0]):
        #     i = np.array(ast.literal_eval(i)) / len(char_to_int)
        #     x[n] = i

        # ys = np.array(ydf.iloc[:, 0])
        # one_hot_encoding = np.zeros((len(char_to_int)))

        # y = np.zeros((len(ydf.iloc[:, 0]), len(one_hot_encoding)))
        # for n, i in enumerate(ys):
        #     temp = one_hot_encoding
        #     temp[ys[n]] = 1
        #     y[n] = temp

        x = self.X[idx:idx + self.batch_size]
        y = self.Y[idx:idx + self.batch_size]

        return (x, y)